Embedding就是利用Embedding模型将本地知识转为向量的过程。
下面的例子将简单的介绍怎么使用。

In [ ]:
import os
from langchain_ollama import OllamaEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

'''
Embedding案例
'''

embed = OllamaEmbeddings(
    model="bge-m3",
    base_url = 'http://localhost:11434'
    # other params...
)

# 嵌入单例文本
input_text = "2025年9月3日，中国将在北京举行抗战胜利80周年纪念活动。"
vector = embed.embed_query(input_text)
# 异步方式
# vector = await embed.aembed_query(input_text)
print(vector[:3])

# 批量文本
input_texts = [input_text, "在此之前，越南、俄罗斯等国也都分别举行了抗战胜利纪念活动。"]
vectors = embed.embed_documents(input_texts)
print(len(vectors))
print(vectors[0][:3])

通过Embedding获取向量后，需要将向量存储起来，以便大模型检索。

In [ ]:
import os
from langchain_ollama import OllamaEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document

embeddings = OllamaEmbeddings(
    model="bge-m3",
    base_url = 'http://localhost:11434'
    # other params...
)

text1 = "2024/2025赛季，利物浦以绝对优势占据了积分榜榜首，它将是这个赛季的英超冠军！"
text2 = "根据2025年的国际形势，预测十年内出现重大军事冲突，或大范围的军事冲突可能行较大，但是发生世界大战的可能性较小。"
text3 = "第23届世界杯将由2026年在加拿大、墨西哥、美国联合举办，是世界杯史上首次由3个国家联合举办同1届赛事。"

def test_local1():
    '''
    Embedding
    '''
    # 如果需要开启模型追踪，可以开启
    # os.environ["LANGSMITH_TRACING"] = "true"
    # os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")

    
    # 创建一个基于内存的向量存储
    vectorstore = InMemoryVectorStore.from_texts(
        [text1, text2, text3],
        embedding = embeddings,
    )
    
    #vectorstore.add_texts()
    
    # 使用向量存储作为检索器
    retriever = vectorstore.as_retriever()

    # 检索文本
    #retrieved_documents = retriever.invoke("哪支俱乐部会是24/25赛季的英超冠军？")
    retrieved_documents = retriever.invoke("最近是否会发生世界大战？")
    # 获取检索到的内容，由于向量库内容较少，只能回答已有的知识
    content = retrieved_documents[0].page_content
    print(content)


def test_local2():
    query_result = embeddings.embed_query(text1)
    print("文本向量长度：", len(query_result), sep='')
    
    doc_results = embeddings.embed_documents(
        [
            "第30轮，利物浦 vs 埃弗顿，比分：3:1",
            "第31轮，富勒姆 vs 利物浦，比分：0:2",
            "第32轮，利物浦 vs 西汉姆联，比分：2:1",
            "第33轮，莱斯特城 vs 利物浦，比分：2:2",
            "第34轮，利物浦 vs 热刺，比分：2:1",
            "第35轮，切尔西 vs 利物浦，比分：0:1",
            "第36轮，利物浦 vs 阿森纳，比分：2:1",
        ])
    print("文本向量数量：", len(doc_results), "，文本向量长度：", len(doc_results[0]), sep='')


def test_local3():
    document_1 = Document(id="1", page_content=text1, metadata={"冠军": "利物浦"})
    document_2 = Document(id="2", page_content=text2, metadata={"世界大战": "可能性较小"})
    document_3 = Document(id="3", page_content=text3)
    documents = [document_1, document_2, document_3]
    
    vectorstore = InMemoryVectorStore.from_texts(
        [text1, text2, text3],
        embedding = embeddings,
    )
    vectorstore.add_documents(documents=documents)
    
    top_n = 10
    for index, (id, doc) in enumerate(vectorstore.store.items()):
        if index < top_n:
            # docs have keys 'id', 'vector', 'text', 'metadata'
            print(f"{id}: {doc['text']}")
        else:
            break
    
    # 删除
    vectorstore.delete(ids=["3"])
    
    # 搜索
    results = vectorstore.similarity_search(query="冠军",k=1)
    for doc in results:
        print(f"* {doc.page_content} [{doc.metadata}]")

    # 解析器
    retriever = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 1, "fetch_k": 2, "lambda_mult": 0.5},
    )
    result = retriever.invoke("冠军")
    print(result)


test_local3()

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy

query = 'apples'
passages = [
        'I like apples', 
        'I like oranges', 
        'Apples and oranges are fruits'
    ]
  
# init embedding model
model_name = 'maidalun1020/bce-embedding-base_v1'
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'batch_size': 64, 'normalize_embeddings': True, 'show_progress_bar': False}

embed_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
  )

# example #1. extract embeddings
query_embedding = embed_model.embed_query(query)
passages_embeddings = embed_model.embed_documents(passages)

# example #2. langchain retriever example
faiss_vectorstore = FAISS.from_texts(passages, embed_model, distance_strategy=DistanceStrategy.MAX_INNER_PRODUCT)

retriever = faiss_vectorstore.as_retriever(search_type="similarity", search_kwargs={"score_threshold": 0.5, "k": 3})

related_passages = retriever.get_relevant_documents(query)

print(related_passages)

参考：https://www.cnblogs.com/longronglang/p/18718487